In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from coffea import util
from coffea.processor import accumulate
import os, sys
import re
import pandas as pd
import hist
import matplotlib as mpl
import mplhep
import utils
from pathlib import Path

sys.path.append("../../../analysisTools/")
import plotTools as ptools

In [2]:
def add_signal_info_to_df(df):
    m1_list = []
    delta_list = []
    ctau_list = []
    
    for point in df.index.values:
        sig_dict = ptools.signalPoint(point)
        m1_list.append(sig_dict['m1'])
        delta_list.append(sig_dict['delta'])
        ctau_list.append(sig_dict['ctau'])
    
    df['m1'] = m1_list
    df['delta'] = delta_list
    df['ctau'] = ctau_list
    
    df = df.sort_values(by=['m1']) # sort by m1

    return df

In [3]:
def save_df_to_csv(df, outdir, outname, isSignal = False):
    Path(outdir).mkdir(parents=True, exist_ok=True)

    if isSignal:
        df = add_signal_info_to_df(df)
    
    df.to_csv(f'{outdir}/{outname}.csv')

    print(f'Saved: {outdir}/{outname}.csv')

# SR BDT-lowLxy-model: Low Lxy (Lxy <= 0.5)

## Background

In [4]:
prefix = './coffea/skimmed/'
suffix = '_LxyBDT-based_LowLxy'

In [5]:
# Background
bkg_files = {
    "QCD": "bkg_2018_QCD_TuneCP5_PSWeights", # No entry for QCD
    "Top": "bkg_2018_Top",
    "DY": "bkg_2018_DY",
    "Diboson": "bkg_2018_Diboson",
    "Triboson": "bkg_2018_Triboson",
    "W+jets": "bkg_2018_WJets",
    "Z+jets": "bkg_2018_ZJets",
}

bkg_histos = {}

for b in bkg_files.keys():
    bkg_histos[b] = util.load(f'{prefix}/{bkg_files[b]}{suffix}.coffea')[0]

In [6]:
# efficiency
pd.options.display.float_format = '{:,.6f}'.format
bkg_eff_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow')

bkg_eff_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT
QCD,0.000002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Top,0.000129,0.000013,0.000008,0.000007,0.000007,0.000007,0.000007,0.000004,0.000000
DY,0.000011,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Diboson,0.000637,0.000044,0.000033,0.000031,0.000030,0.000030,0.000027,0.000012,0.000000
Triboson,0.001933,0.000158,0.000104,0.000092,0.000091,0.000091,0.000076,0.000042,0.000000
W+jets,0.002310,0.000173,0.000139,0.000132,0.000131,0.000131,0.000114,0.000048,0.000000
Z+jets,0.006645,0.000398,0.000325,0.000310,0.000309,0.000309,0.000292,0.000120,0.000000
Total,0.000019,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
outdir = './csv/Cutflow/Efficiency/'
outname = 'SR_LxyBDT-based_Background_Efficiency_LowLxyBin'

df_to_save = bkg_eff_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/Efficiency//SR_LxyBDT-based_Background_Efficiency_LowLxyBin.csv


In [8]:
# weighted event count
pd.options.display.float_format = '{:,.2f}'.format
bkg_cts_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_cts')

bkg_cts_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT
QCD,"38,039.83","1,882.67","1,326.86","1,319.01","1,319.01","1,318.41",632.25,624.41,0.00
Top,"5,667.49",559.22,349.76,314.03,312.99,312.95,294.00,154.46,0.53
DY,"14,495.72",671.44,623.25,623.25,623.25,623.25,455.65,407.47,0.00
Diboson,"4,424.20",307.91,227.24,213.36,211.27,210.33,189.35,86.23,0.29
Triboson,53.00,4.32,2.85,2.51,2.51,2.50,2.09,1.16,0.00
W+jets,"227,212.60","17,009.81","13,645.63","12,937.21","12,911.49","12,907.15","11,204.39","4,750.18",16.56
Z+jets,"142,220.15","8,524.55","6,957.67","6,625.48","6,613.11","6,612.30","6,256.76","2,562.32",6.39
Total,"432,113.00","28,959.92","23,133.27","22,034.86","21,993.63","21,986.91","19,034.50","8,586.23",23.77


In [9]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = 'SR_LxyBDT-based_Background_EventCount_Weighted_LowLxyBin'

df_to_save = bkg_cts_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_LxyBDT-based_Background_EventCount_Weighted_LowLxyBin.csv


In [10]:
# raw event count
pd.options.display.float_format = '{:,.0f}'.format
bkg_cts_raw_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_nevts')

bkg_cts_raw_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT
QCD,12311,381,160,159,159,158,3,2,0
Top,49818,4688,3058,2348,2337,2336,2006,1034,3
DY,205,13,11,11,11,11,9,7,0
Diboson,18067,1234,915,860,852,848,767,348,1
Triboson,2257,170,116,106,105,104,92,42,0
W+jets,697657,48425,37015,35472,35415,35377,29090,12496,45
Z+jets,1281459,67427,52627,50986,50934,50903,44480,18372,48
Total,2061774,122338,93902,89942,89813,89737,76447,32301,97


In [11]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = 'SR_LxyBDT-based_Background_EventCount_Raw_LowLxyBin'

df_to_save = bkg_cts_raw_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/EventCount_Raw//SR_LxyBDT-based_Background_EventCount_Raw_LowLxyBin.csv


In [12]:
nBkg = bkg_cts_df.loc['Total'][-1]

print(f'Total number of backgrounds (weighted) after the last SR cut: {nBkg:.2f}')

Total number of backgrounds (weighted) after the last SR cut: 23.77


## Signal

In [13]:
prefix = './coffea/skimmed/'
suffix = '_skimmed'

In [14]:
# Signal
sig_histo = util.load(f'{prefix}/LxyBDT-based_LowLxy.coffea')[0]

In [15]:
si = utils.get_signal_point_dict(sig_histo)

pd.options.display.float_format = '{:,.1f}'.format
si

,mchi,dmchi,ctau,m1,m2,delta,name
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,5.5,1.0,1.0,5.0,6.0,0.2,sig_2018_Mchi-5p5_dMchi-1p0_ctau-1
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,5.5,1.0,100.0,5.0,6.0,0.2,sig_2018_Mchi-5p5_dMchi-1p0_ctau-100
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.5,1.0,10.0,5.0,6.0,0.2,sig_2018_Mchi-5p5_dMchi-1p0_ctau-10
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,5.2,0.5,100.0,5.0,5.5,0.1,sig_2018_Mchi-5p25_dMchi-0p5_ctau-100
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,55.0,10.0,10.0,50.0,60.0,0.2,sig_2018_Mchi-55p0_dMchi-10p0_ctau-10
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,5.2,0.5,1.0,5.0,5.5,0.1,sig_2018_Mchi-5p25_dMchi-0p5_ctau-1
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,55.0,10.0,100.0,50.0,60.0,0.2,sig_2018_Mchi-55p0_dMchi-10p0_ctau-100
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5.2,0.5,10.0,5.0,5.5,0.1,sig_2018_Mchi-5p25_dMchi-0p5_ctau-10
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,55.0,10.0,1.0,50.0,60.0,0.2,sig_2018_Mchi-55p0_dMchi-10p0_ctau-1
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,52.5,5.0,100.0,50.0,55.0,0.1,sig_2018_Mchi-52p5_dMchi-5p0_ctau-100


In [16]:
cutnames = utils.get_signal_list_of_cuts(sig_histo)

cutnames

['Preselections',
 '0 < n(jet) < 3 & n(good vertex) > 0',
 'No b-tagged jets',
 'Leading jet |eta| < 2.4',
 'Leading jet pT > 80 GeV',
 'dPhi(MET,leading jet) > 1.5',
 'dPhi(MET,all jets) > 0.75',
 'Lxy <= 0.5',
 'BDT']

### Efficiency

In [17]:
pd.options.display.float_format = '{:,.6f}'.format

In [18]:
eff_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow')

In [19]:
table = pd.DataFrame.from_dict(eff_cutflow, orient='index')
table.columns = cutnames

In [20]:
ct = 1.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,0.014484,0.001842,0.001519,0.001421,0.001418,0.001418,0.001404,0.000489,0.000211
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,0.014782,0.002143,0.001894,0.001811,0.001811,0.001811,0.001794,0.000635,0.000187
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,0.033826,0.006982,0.005961,0.005695,0.005695,0.005695,0.005651,0.004049,0.002715
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,0.036318,0.006259,0.005421,0.005217,0.005217,0.005217,0.005197,0.003437,0.002009
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,0.026904,0.005099,0.004306,0.004165,0.004161,0.004161,0.004135,0.002833,0.001869
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,0.030555,0.005051,0.004338,0.004133,0.004127,0.004127,0.004103,0.002508,0.001520
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,0.020878,0.003848,0.003369,0.003242,0.003235,0.003235,0.003195,0.002101,0.001438
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,0.016920,0.002932,0.002531,0.002386,0.002386,0.002386,0.002357,0.001342,0.000863
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,0.023214,0.003481,0.002980,0.002881,0.002881,0.002881,0.002855,0.001618,0.000909
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,0.018942,0.002826,0.002381,0.002275,0.002273,0.002273,0.002253,0.001095,0.000575


In [21]:
outdir = './csv/Cutflow/Efficiency/'
outname = 'SR_LxyBDT-based_Signal_ctau-1_samples_Efficiency_LowLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/Efficiency//SR_LxyBDT-based_Signal_ctau-1_samples_Efficiency_LowLxyBin.csv


In [22]:
ct = 10.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,0.010930,0.001187,0.000987,0.000935,0.000931,0.000931,0.000876,0.000238,0.000021
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,0.031125,0.005252,0.004562,0.004329,0.004329,0.004329,0.004260,0.001484,0.000819
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,0.010676,0.001022,0.000847,0.000792,0.000792,0.000792,0.000771,0.000218,0.000019
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,0.032445,0.004564,0.003903,0.003752,0.003752,0.003752,0.003708,0.001272,0.000498
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,0.024702,0.003809,0.003226,0.003090,0.003090,0.003090,0.003027,0.001005,0.000466
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,0.025875,0.003520,0.003022,0.002902,0.002902,0.002902,0.002875,0.000921,0.000345
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,0.019309,0.002550,0.002202,0.002122,0.002120,0.002120,0.002075,0.000647,0.000323
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,0.020128,0.002757,0.002373,0.002303,0.002301,0.002301,0.002280,0.000654,0.000186
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,0.016286,0.001989,0.001734,0.001669,0.001669,0.001669,0.001649,0.000520,0.000130
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,0.015838,0.001936,0.001685,0.001584,0.001584,0.001584,0.001549,0.000411,0.000147


In [23]:
outdir = './csv/Cutflow/Efficiency/'
outname = 'SR_LxyBDT-based_Signal_ctau-10_samples_Efficiency_HighLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/Efficiency//SR_LxyBDT-based_Signal_ctau-10_samples_Efficiency_HighLxyBin.csv


In [24]:
ct = 100.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,0.007414,0.000647,0.000562,0.000528,0.000528,0.000528,0.000501,0.000150,0.000003
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,0.007306,0.000596,0.000504,0.000475,0.000475,0.000475,0.000457,0.000180,0.000010
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,0.022439,0.002420,0.002063,0.001980,0.001980,0.001980,0.001893,0.000485,0.000077
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,0.022522,0.002231,0.001905,0.001847,0.001847,0.001847,0.001804,0.000464,0.000059
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,0.017576,0.001802,0.001476,0.001411,0.001411,0.001411,0.001364,0.000319,0.000047
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,0.017664,0.001657,0.001419,0.001371,0.001371,0.001371,0.001331,0.000363,0.000033
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,0.013585,0.001285,0.001082,0.001052,0.001052,0.001052,0.001011,0.000238,0.000033
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,0.013536,0.001197,0.001041,0.000995,0.000995,0.000995,0.000966,0.000284,0.000014
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,0.010335,0.000922,0.000793,0.000760,0.000760,0.000760,0.000722,0.000205,0.000030
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,0.010649,0.000928,0.000818,0.000775,0.000775,0.000775,0.000749,0.000238,0.000010


In [25]:
outdir = './csv/Cutflow/Efficiency/'
outname = 'SR_LxyBDT-based_Signal_ctau-100_samples_Efficiency_LowLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/Efficiency//SR_LxyBDT-based_Signal_ctau-100_samples_Efficiency_LowLxyBin.csv


### Weighted Event count

In [26]:
pd.options.display.float_format = '{:,.2f}'.format

In [27]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')

In [28]:
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [29]:
ct = 1.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,"2,748.24",349.57,288.31,269.62,268.98,268.98,266.40,92.86,39.99
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,"62,382.76","9,045.04","7,993.01","7,642.38","7,642.38","7,642.38","7,572.31","2,681.89",789.00
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,205.19,42.35,36.16,34.55,34.55,34.55,34.28,24.56,16.47
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,"6,562.28","1,130.93",979.54,942.72,942.72,942.72,939.12,620.99,363.08
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,512.94,97.21,82.10,79.40,79.34,79.34,78.84,54.00,35.63
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,"16,193.18","2,676.90","2,298.73","2,190.27","2,187.08","2,187.08","2,174.33","1,328.88",805.66
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,"49,723.24","9,164.66","8,024.74","7,722.24","7,704.44","7,704.44","7,609.52","5,003.17","3,424.65"
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,995.45,172.47,148.91,140.36,140.36,140.36,138.67,78.94,50.76
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,"33,794.92","5,067.47","4,338.18","4,193.56","4,193.56","4,193.56","4,156.59","2,355.36","1,323.75"
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,"30,374.51","4,531.00","3,817.78","3,648.14","3,644.60","3,644.60","3,612.88","1,755.18",921.56


In [30]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = 'SR_LxyBDT-based_Signal_ctau-1_samples_EventCount_Weighted_LowLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_LxyBDT-based_Signal_ctau-1_samples_EventCount_Weighted_LowLxyBin.csv


In [31]:
# without m1 = 30 GeV
ct = 1.0
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,"2,748.24",349.57,288.31,269.62,268.98,268.98,266.40,92.86,39.99
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,"62,382.76","9,045.04","7,993.01","7,642.38","7,642.38","7,642.38","7,572.31","2,681.89",789.00
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,205.19,42.35,36.16,34.55,34.55,34.55,34.28,24.56,16.47
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,"6,562.28","1,130.93",979.54,942.72,942.72,942.72,939.12,620.99,363.08
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,512.94,97.21,82.10,79.40,79.34,79.34,78.84,54.00,35.63
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,"16,193.18","2,676.90","2,298.73","2,190.27","2,187.08","2,187.08","2,174.33","1,328.88",805.66
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,995.45,172.47,148.91,140.36,140.36,140.36,138.67,78.94,50.76
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,"30,374.51","4,531.00","3,817.78","3,648.14","3,644.60","3,644.60","3,612.88","1,755.18",921.56
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,"39,395.49","5,811.03","5,006.12","4,654.25","4,648.89","4,648.89","4,616.98","1,892.76",778.60
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,"1,394.26",207.15,178.78,170.89,170.89,170.89,169.82,77.43,38.40


In [32]:
ct = 10.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,211.62,22.98,19.10,18.10,18.03,18.03,16.97,4.61,0.40
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,18.96,3.20,2.78,2.64,2.64,2.64,2.60,0.90,0.50
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,"6,090.24",583.27,483.38,451.73,451.73,451.73,439.55,124.17,10.95
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,625.95,88.06,75.30,72.38,72.38,72.38,71.54,24.54,9.60
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,47.30,7.30,6.18,5.92,5.92,5.92,5.80,1.93,0.89
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,"1,569.13",213.48,183.29,176.01,176.01,176.01,174.34,55.84,20.95
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,"15,555.45","2,054.32","1,773.84","1,709.70","1,707.69","1,707.69","1,671.59",521.04,260.53
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,"27,263.50","3,734.67","3,214.54","3,119.89","3,117.26","3,117.26","3,088.33",886.02,252.38
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,"3,043.72",371.80,324.16,312.00,312.00,312.00,308.26,97.13,24.27
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,94.00,11.49,10.00,9.40,9.40,9.40,9.20,2.44,0.87


In [33]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = 'SR_LxyBDT-based_Signal_ctau-10_samples_EventCount_Weighted_LowLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_LxyBDT-based_Signal_ctau-10_samples_EventCount_Weighted_LowLxyBin.csv


In [34]:
# without m1 = 30 GeV
ct = 10.0
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,211.62,22.98,19.10,18.10,18.03,18.03,16.97,4.61,0.40
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,18.96,3.20,2.78,2.64,2.64,2.64,2.60,0.90,0.50
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,"6,090.24",583.27,483.38,451.73,451.73,451.73,439.55,124.17,10.95
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,625.95,88.06,75.30,72.38,72.38,72.38,71.54,24.54,9.60
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,47.30,7.30,6.18,5.92,5.92,5.92,5.80,1.93,0.89
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,"1,569.13",213.48,183.29,176.01,176.01,176.01,174.34,55.84,20.95
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,"3,043.72",371.80,324.16,312.00,312.00,312.00,308.26,97.13,24.27
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,94.00,11.49,10.00,9.40,9.40,9.40,9.20,2.44,0.87
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,118.48,13.27,10.81,10.19,10.19,10.19,9.84,2.64,0.55
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,"3,591.89",398.72,340.80,322.41,322.41,322.41,313.43,81.74,10.33


In [35]:
ct = 100.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,14.29,1.25,1.08,1.02,1.02,1.02,0.96,0.29,0.01
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,443.66,36.19,30.61,28.82,28.82,28.82,27.73,10.95,0.58
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,1.37,0.15,0.13,0.12,0.12,0.12,0.12,0.03,0.00
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,43.76,4.34,3.70,3.59,3.59,3.59,3.50,0.90,0.12
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,3.35,0.34,0.28,0.27,0.27,0.27,0.26,0.06,0.01
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,108.83,10.21,8.74,8.45,8.45,8.45,8.20,2.23,0.20
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,"1,302.77",123.25,103.80,100.85,100.85,100.85,96.92,22.86,3.20
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,"24,707.11","2,184.60","1,899.41","1,816.96","1,816.96","1,816.96","1,763.83",518.69,25.75
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,6.14,0.55,0.47,0.45,0.45,0.45,0.43,0.12,0.02
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,200.85,17.50,15.42,14.61,14.61,14.61,14.12,4.49,0.20


In [36]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = 'SR_LxyBDT-based_Signal_ctau-100_samples_EventCount_Weighted_LowLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_LxyBDT-based_Signal_ctau-100_samples_EventCount_Weighted_LowLxyBin.csv


In [37]:
# without m1 = 30 GeV
ct = 100.0
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,14.29,1.25,1.08,1.02,1.02,1.02,0.96,0.29,0.01
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,443.66,36.19,30.61,28.82,28.82,28.82,27.73,10.95,0.58
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,1.37,0.15,0.13,0.12,0.12,0.12,0.12,0.03,0.00
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,43.76,4.34,3.70,3.59,3.59,3.59,3.50,0.90,0.12
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,3.35,0.34,0.28,0.27,0.27,0.27,0.26,0.06,0.01
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,108.83,10.21,8.74,8.45,8.45,8.45,8.20,2.23,0.20
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,6.14,0.55,0.47,0.45,0.45,0.45,0.43,0.12,0.02
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,200.85,17.50,15.42,14.61,14.61,14.61,14.12,4.49,0.20
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,7.60,0.57,0.48,0.46,0.46,0.46,0.44,0.13,0.00
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,238.69,19.21,16.19,15.29,15.29,15.29,14.98,4.96,0.17


### Raw count

In [38]:
pd.options.display.float_format = '{:,.0f}'.format

In [39]:
raw_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_nevts')

In [40]:
table = pd.DataFrame.from_dict(raw_cutflow, orient='index')
table.columns = cutnames

In [41]:
ct = 1.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,4261,542,447,418,417,417,413,144,62
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,3558,516,456,436,436,436,432,153,45
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,761,157,134,128,128,128,127,91,61
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,14604,2517,2180,2098,2098,2098,2090,1382,808
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,8179,1550,1309,1266,1265,1265,1257,861,568
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,10151,1678,1441,1373,1371,1371,1363,833,505
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,8377,1544,1352,1301,1298,1298,1282,843,577
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,7098,1230,1062,1001,1001,1001,989,563,362
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,10056,1508,1291,1248,1248,1248,1237,701,394
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,8600,1283,1081,1033,1032,1032,1023,497,261


In [42]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = 'SR_LxyBDT-based_Signal_ctau-1_samples_EventCount_Raw_LowLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Raw//SR_LxyBDT-based_Signal_ctau-1_samples_EventCount_Raw_LowLxyBin.csv


In [43]:
ct = 10.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,3168,344,286,271,270,270,254,69,6
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,3609,609,529,502,502,502,494,172,95
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5002,479,397,371,371,371,361,102,9
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,13492,1898,1623,1560,1560,1560,1542,529,207
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,9065,1398,1184,1134,1134,1134,1111,369,171
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,10341,1407,1208,1160,1160,1160,1149,368,138
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,7762,1025,885,853,852,852,834,260,130
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,10374,1421,1223,1187,1186,1186,1175,337,96
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,6515,796,694,668,668,668,660,208,52
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,8280,1012,881,828,828,828,810,215,77


In [44]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = 'SR_LxyBDT-based_Signal_ctau-10_samples_EventCount_Raw_LowLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Raw//SR_LxyBDT-based_Signal_ctau-10_samples_EventCount_Raw_LowLxyBin.csv


In [45]:
ct = 100.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,2177,190,165,155,155,155,147,44,1
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,10655,869,735,692,692,692,666,263,14
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,6993,754,643,617,617,617,590,151,24
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,15529,1539,1314,1274,1274,1274,1244,320,41
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,10574,1084,888,849,849,849,821,192,28
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,16226,1522,1303,1259,1259,1259,1222,333,30
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,5296,501,422,410,410,410,394,93,13
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,14389,1272,1106,1058,1058,1058,1027,302,15
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,6241,557,479,459,459,459,436,124,18
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,8195,714,629,596,596,596,576,183,8


In [46]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = 'SR_LxyBDT-based_Signal_ctau-100_samples_EventCount_Raw_LowLxyBin'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Raw//SR_LxyBDT-based_Signal_ctau-100_samples_EventCount_Raw_LowLxyBin.csv


In [47]:
# for a given ctau and mass splitting
ct = 1.0
delta = 0.1

points = (si["ctau"] == ct) & (si["delta"] == delta)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",Lxy <= 0.5,BDT
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,3558,516,456,436,436,436,432,153,45
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,14604,2517,2180,2098,2098,2098,2090,1382,808
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,10151,1678,1441,1373,1371,1371,1363,833,505
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,10056,1508,1291,1248,1248,1248,1237,701,394
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,8600,1283,1081,1033,1032,1032,1023,497,261
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,7390,1090,939,873,872,872,866,355,146
Total,54359,8592,7388,7061,7057,7057,7011,3921,2159


## S/sqrt(B)

In [48]:
nBkg

23.770825624758743

In [49]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [50]:
pd.options.display.float_format = '{:,.4f}'.format

In [51]:
df = pd.DataFrame(table.loc[:, cutnames[-1]]) # last cut

m1_list = []
delta_list = []
ctau_list = []

for point in df.index.values:
    sig_dict = ptools.signalPoint(point)
    m1_list.append(sig_dict['m1'])
    delta_list.append(sig_dict['delta'])
    ctau_list.append(sig_dict['ctau'])

df['m1'] = m1_list
df['delta'] = delta_list
df['ctau'] = ctau_list

df = df.sort_values(by=['m1']) # sort by m1

In [52]:
df = df.rename(columns={df.columns[0]: 'Signal'})

df['Background'] = nBkg
df

,Signal,m1,delta,ctau,Background
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,39.9874,5.0000,0.2000,1.0000,23.7708
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,0.0066,5.0000,0.2000,100.0000,23.7708
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,0.3998,5.0000,0.2000,10.0000,23.7708
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,0.5832,5.0000,0.1000,100.0000,23.7708
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,788.9991,5.0000,0.1000,1.0000,23.7708
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,10.9529,5.0000,0.1000,10.0000,23.7708
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,0.1737,10.0000,0.1000,100.0000,23.7708
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,10.3336,10.0000,0.1000,10.0000,23.7708
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,0.5548,10.0000,0.2000,10.0000,23.7708
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,0.0039,10.0000,0.2000,100.0000,23.7708


In [53]:
df['S/sqrt(B)'] = df['Signal'] / np.sqrt(df['Background'])

df = df[['m1', 'delta', 'ctau', 'Signal', 'Background', 'S/sqrt(B)']]
df

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,5.0000,0.2000,1.0000,39.9874,23.7708,8.2016
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,5.0000,0.2000,100.0000,0.0066,23.7708,0.0013
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.0000,0.2000,10.0000,0.3998,23.7708,0.0820
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,5.0000,0.1000,100.0000,0.5832,23.7708,0.1196
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,5.0000,0.1000,1.0000,788.9991,23.7708,161.8283
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5.0000,0.1000,10.0000,10.9529,23.7708,2.2465
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,10.0000,0.1000,100.0000,0.1737,23.7708,0.0356
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,10.0000,0.1000,10.0000,10.3336,23.7708,2.1195
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,10.0000,0.2000,10.0000,0.5548,23.7708,0.1138
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,10.0000,0.2000,100.0000,0.0039,23.7708,0.0008


In [54]:
outdir = './csv/SR_Yields/'
outname = 'SR_LxyBDT-based_LowLxy_S_over_B'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/SR_Yields//SR_LxyBDT-based_LowLxy_S_over_B.csv


### ctau = 1mm signal points

In [55]:
# S/sqrt(B) in ctau = 1mm, delta = 0.1
ct = 1.0
delta = 0.1

df[ (df.ctau == ct) & (df.delta == delta) ]

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,5.0000,0.1000,1.0000,788.9991,23.7708,161.8283
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,10.0000,0.1000,1.0000,778.6015,23.7708,159.6956
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,20.0000,0.1000,1.0000,921.5599,23.7708,189.0172
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,30.0000,0.1000,1.0000,"1,323.7455",23.7708,271.5078
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,40.0000,0.1000,1.0000,805.6649,23.7708,165.2465
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,50.0000,0.1000,1.0000,363.0769,23.7708,74.4692


In [56]:
# S/sqrt(B) in ctau = 1mm, delta = 0.2
ct = 1.0
delta = 0.2

df[ (df.ctau == ct) & (df.delta == delta) ]

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,5.0000,0.2000,1.0000,39.9874,23.7708,8.2016
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,10.0000,0.2000,1.0000,38.3965,23.7708,7.8753
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,20.0000,0.2000,1.0000,50.7648,23.7708,10.4122
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,30.0000,0.2000,1.0000,"3,424.6533",23.7708,702.4161
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,40.0000,0.2000,1.0000,35.6273,23.7708,7.3074
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,50.0000,0.2000,1.0000,16.4687,23.7708,3.3778


### ctau = 10mm signal points

In [57]:
# S/sqrt(B) in ctau = 10mm, delta = 0.1
ct = 10.0
delta = 0.1

df[ (df.ctau == ct) & (df.delta == delta) ]

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5.0000,0.1000,10.0000,10.9529,23.7708,2.2465
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,10.0000,0.1000,10.0000,10.3336,23.7708,2.1195
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,20.0000,0.1000,10.0000,24.2694,23.7708,4.9778
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,30.0000,0.1000,10.0000,252.3803,23.7708,51.7647
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,40.0000,0.1000,10.0000,20.9482,23.7708,4.2966
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,50.0000,0.1000,10.0000,9.6048,23.7708,1.9700


In [58]:
# S/sqrt(B) in ctau = 10mm, delta = 0.2
ct = 10.0
delta = 0.2

df[ (df.ctau == ct) & (df.delta == delta) ]

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.0000,0.2000,10.0000,0.3998,23.7708,0.0820
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,10.0000,0.2000,10.0000,0.5548,23.7708,0.1138
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,20.0000,0.2000,10.0000,0.8744,23.7708,0.1793
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,30.0000,0.2000,10.0000,260.5250,23.7708,53.4352
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,40.0000,0.2000,10.0000,0.8920,23.7708,0.1830
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,50.0000,0.2000,10.0000,0.4993,23.7708,0.1024


### ctau = 100mm signal points

In [59]:
# S/sqrt(B) in ctau = 100mm, delta = 0.1
ct = 100.0
delta = 0.1

df[ (df.ctau == ct) & (df.delta == delta) ]

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,5.0000,0.1000,100.0000,0.5832,23.7708,0.1196
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,10.0000,0.1000,100.0000,0.1737,23.7708,0.0356
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,20.0000,0.1000,100.0000,0.1964,23.7708,0.0403
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,30.0000,0.1000,100.0000,25.7493,23.7708,5.2813
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,40.0000,0.1000,100.0000,0.2010,23.7708,0.0412
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,50.0000,0.1000,100.0000,0.1155,23.7708,0.0237


In [60]:
# S/sqrt(B) in ctau = 100mm, delta = 0.2
ct = 100.0
delta = 0.2

df[ (df.ctau == ct) & (df.delta == delta) ]

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,5.0000,0.2000,100.0000,0.0066,23.7708,0.0013
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,10.0000,0.2000,100.0000,0.0039,23.7708,0.0008
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,20.0000,0.2000,100.0000,0.0177,23.7708,0.0036
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,30.0000,0.2000,100.0000,3.1970,23.7708,0.6557
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,40.0000,0.2000,100.0000,0.0089,23.7708,0.0018
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,50.0000,0.2000,100.0000,0.0047,23.7708,0.0010
